In [1]:
#Project 1

In [2]:
#importing the necessities
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [3]:
#login info
mysql_args = {
    "uid" : "root",
    "pwd" : "even5Keeled$",
    "hostname" : "localhost",
    "dbname" : "sakila_dw2"
}

mongodb_args = {
    "user_name" : "edwyzhang",
    "password" : "IFNXmFLUY2sYMw9j",
    "cluster_name" : "taketwocluster",
    "cluster_subnet" : "ozwxv",
    "cluster_location" : "atlas", # "local"
    "db_name" : "sakila"
}

In [4]:
#code for sql operations
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [5]:
#client info
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd())

json_files = {"customer" : 'customer.json',
              "payment" : 'payment.json',
              "store" : 'store.json'
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)         

In [6]:
#connecting to create databases
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "even5Keeled$"

src_dbname = "sakila"
dst_dbname = "sakila_dw2"

conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

connection.close()

In [7]:
#creating dim_date
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe
    
def set_df(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()
    
sql_dim_date = "SELECT date_key, full_date FROM sakila.dim_date;"
df_dim_date = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_date)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
set_df(user_id, pwd, host_name, dst_dbname, df_dim_date, 'dim_date', 'date_key', "insert")

In [8]:
#taking in from csv
data_dir = os.path.join(os.getcwd())
data_file = os.path.join(data_dir, 'customer.csv')

df_customer = pd.read_csv(data_file, header=0, index_col=0)
df_customer.head()

,store_id,first_name,last_name,email,address_id,active,create_date,last_update
customer_id,,,,,,,,
1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20
3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20
4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20
5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [9]:
df_customer.drop(['address_id','active','last_update'], axis=1, inplace=True)
df_customer.columns

Index(['store_id', 'first_name', 'last_name', 'email', 'create_date'], dtype='object')

In [10]:
#taking in from mongodb
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "payment"

df_payment = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_payment.drop(['rental_id', 'staff_id', 'last_update'], axis=1, inplace=True)

df_payment.head(2)

,payment_id,customer_id,amount,payment_date
0,1,1,2.99,2005-05-25 11:30:37
1,2,1,0.99,2005-05-28 10:35:23


In [11]:
#taking in info from third entity
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "store"

df_store = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_store.drop(['manager_staff_id'], axis=1, inplace=True)

df_store.head(2)

,store_id,address_id,last_update
0,1,1,2006-02-15 04:57:12
1,2,2,2006-02-15 04:57:12


In [12]:
#adding keys
df_store.insert(0, "store_key", range(1, df_store.shape[0]+1))
df_payment.insert(0, "payment_key", range(1, df_payment.shape[0]+1))
df_customer.insert(0, "customer_key", range(1, df_customer.shape[0]+1))

df_customer.head(2)

,customer_key,store_id,first_name,last_name,email,create_date
customer_id,,,,,,
1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,2006-02-14 22:04:36
2,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,2006-02-14 22:04:36


In [13]:
df_payment.head(2)

,payment_key,payment_id,customer_id,amount,payment_date
0,1,1,1,2.99,2005-05-25 11:30:37
1,2,2,1,0.99,2005-05-28 10:35:23


In [14]:
df_store.head(2)

,store_key,store_id,address_id,last_update
0,1,1,1,2006-02-15 04:57:12
1,2,2,2,2006-02-15 04:57:12


In [15]:
#making table
db_operation = "insert"

tables = [('dim_customers', df_customer, 'customer_key'),
          ('dim_payments', df_payment, 'payment_key'),
         ('dim_stores', df_store, 'store_key')]
for table_name, dataframe, primary_key in tables:
    set_df(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [16]:
#merging tables
df_fact_table = pd.merge(df_customer, df_payment, on='customer_id', how='inner')
df_fact_table.head(2)

,customer_id,customer_key,store_id,first_name,last_name,email,create_date,payment_key,payment_id,amount,payment_date
0,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,2006-02-14 22:04:36,1,1,2.99,2005-05-25 11:30:37
1,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,2006-02-14 22:04:36,2,2,0.99,2005-05-28 10:35:23


In [17]:
df_fact_table = pd.merge(df_fact_table, df_store, on='store_id', how='inner')
df_fact_table.head(2)

,customer_id,customer_key,store_id,first_name,last_name,email,create_date,payment_key,payment_id,amount,payment_date,store_key,address_id,last_update
0,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,2006-02-14 22:04:36,1,1,2.99,2005-05-25 11:30:37,1,1,2006-02-15 04:57:12
1,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,2006-02-14 22:04:36,2,2,0.99,2005-05-28 10:35:23,1,1,2006-02-15 04:57:12


In [18]:
#perform lookup operations
df_dim_create_date = df_dim_date.rename(columns={"date_key" : "create_date_key", "full_date" : "create_date"})
df_fact_table.create_date = df_fact_table.create_date.astype('datetime64[ns]').dt.date
df_fact_table = pd.merge(df_fact_table, df_dim_create_date, on='create_date', how='left')
df_fact_table.drop(['create_date'], axis=1, inplace=True)
df_fact_table.head(2)

,customer_id,customer_key,store_id,first_name,last_name,email,payment_key,payment_id,amount,payment_date,store_key,address_id,last_update,create_date_key
0,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,1,1,2.99,2005-05-25 11:30:37,1,1,2006-02-15 04:57:12,20060214
1,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,2,2,0.99,2005-05-28 10:35:23,1,1,2006-02-15 04:57:12,20060214


In [19]:
df_dim_create_date.head(2)

,create_date_key,create_date
0,20050101,2005-01-01
1,20050102,2005-01-02


In [20]:
#perform lookup operations
df_dim_payment_date = df_dim_date.rename(columns={"date_key" : "payment_date_key", "full_date" : "payment_date"})
df_fact_table.payment_date = df_fact_table.payment_date.astype('datetime64[ns]').dt.date
df_fact_table = pd.merge(df_fact_table, df_dim_payment_date, on='payment_date', how='left')
df_fact_table.drop(['payment_date'], axis=1, inplace=True)
df_fact_table.head(2)

,customer_id,customer_key,store_id,first_name,last_name,email,payment_key,payment_id,amount,store_key,address_id,last_update,create_date_key,payment_date_key
0,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,1,1,2.99,1,1,2006-02-15 04:57:12,20060214,20050525
1,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,2,2,0.99,1,1,2006-02-15 04:57:12,20060214,20050528


In [21]:
#transformations to fact table
df_fact_table.drop(['customer_id'], axis=1, inplace=True)
df_fact_table.drop(['payment_id'], axis=1, inplace=True)
df_fact_table.drop(['address_id'], axis=1, inplace=True)
df_fact_table.drop(['store_id'], axis=1, inplace=True)
df_fact_table.head(2)

,customer_key,first_name,last_name,email,payment_key,amount,store_key,last_update,create_date_key,payment_date_key
0,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,1,2.99,1,2006-02-15 04:57:12,20060214,20050525
1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,2,0.99,1,2006-02-15 04:57:12,20060214,20050528


In [22]:
#inserting primary key
df_fact_table.insert(0, "fact_table_key", range(1, df_fact_table.shape[0]+1))
df_fact_table.head(2)

,fact_table_key,customer_key,first_name,last_name,email,payment_key,amount,store_key,last_update,create_date_key,payment_date_key
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,1,2.99,1,2006-02-15 04:57:12,20060214,20050525
1,2,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,2,0.99,1,2006-02-15 04:57:12,20060214,20050528


In [23]:
#transformations
ordered_columns = ['fact_table_key','customer_key'
                   ,'payment_key','store_key'
                   ,'first_name','last_name','email',
                  'amount', 'last_update', 'create_date_key'
                  ,'payment_date_key']

df_fact_table = df_fact_table[ordered_columns]
df_fact_table.head(2)

,fact_table_key,customer_key,payment_key,store_key,first_name,last_name,email,amount,last_update,create_date_key,payment_date_key
0,1,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,2.99,2006-02-15 04:57:12,20060214,20050525
1,2,1,2,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,0.99,2006-02-15 04:57:12,20060214,20050528


In [24]:
#write into mySQL
dataframe = df_fact_table
table_name = 'fact_table'
primary_key = 'fact_table_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [25]:
#verifying
df_fact_table.head(2)

,fact_table_key,customer_key,payment_key,store_key,first_name,last_name,email,amount,last_update,create_date_key,payment_date_key
0,1,1,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,2.99,2006-02-15 04:57:12,20060214,20050525
1,2,1,2,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,0.99,2006-02-15 04:57:12,20060214,20050528


In [26]:
#query with fact table
sql_fact_table = """
select p.customer_id as 'Customer ID'
, format(sum(p.amount),0) as 'Total Spent'
from sakila_dw2.fact_table as ft
inner join sakila_dw2.dim_payments as p
on ft.payment_key = p.payment_key
group by p.customer_id
order by 'Customer ID' desc;
"""

In [27]:
df_fact_table = get_sql_dataframe(sql_fact_table, **mysql_args)
df_fact_table

,Customer ID,Total Spent
0,1,119
1,2,129
2,3,136
3,4,82
4,5,145
5,6,94
6,7,152
7,8,93
8,9,90
9,10,100


In [28]:
#query with dim_customers
sql_customers = """
select store_id,
count(*) as number_of_customers
from dim_customers
    group by store_id
    order by store_id;
"""

In [29]:
df_dim_customers = get_sql_dataframe(sql_customers, **mysql_args)
df_dim_customers

,store_id,number_of_customers
0,1,326
1,2,273


In [30]:
#query with dim_payments
sql_payments = """
select customer_id,
count(amount) as under_five_item
from dim_payments
    where amount < 5
    group by customer_id
    order by customer_id;
"""

In [31]:
df_dim_payments = get_sql_dataframe(sql_payments, **mysql_args)
df_dim_payments

,customer_id,under_five_item
0,1,26
1,2,16
2,3,16
3,4,17
4,5,32
5,6,22
6,7,20
7,8,19
8,9,19
9,10,21
